# Manage Imports

In [1]:
import numpy as np
import gym
import random

# Initialize Environment

In [2]:
env = gym.make("FrozenLake-v0")

# Create Q-Table

Create an $M \times N$ matrix, with 
- $M$ = size of action space
- $N$ = size of state space / observation space

In [3]:
action_space_size = env.action_space.n
observation_space_size = env.observation_space.n

qtable = np.zeros((observation_space_size, action_space_size))
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


# Setup Hyper Parameters

In [4]:
total_episodes = 15000
learning_rate = 0.8
max_steps = 99

#discounting rate
gamma = 0.95

#exploration rate
epsilon = 1.0
max_epsilon = 1.0
min_epsilon = 0.01

#exponential decay rate for exploration probability
decay_rate = 0.005

# The Q-Learning Algorithm